In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(model="gemma2-9b-it")

response = llm.invoke("What is the capital of Pakistan?")

print(response.content)


The capital of Pakistan is **Islamabad**. 



In [ ]:
import os
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") 

# Initialize the LLM
llm = ChatGroq(
    model_name="gemma2-9b-it",  
    temperature=0,  
)

def get_sql_type(dtype):
    """
    Map pandas dtype to SQL type.
    
    Args:
        dtype: The pandas dtype to map.
        
    Returns:
        str: The corresponding SQL type.
    """
    if pd.api.types.is_integer_dtype(dtype):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(dtype):
        return "REAL"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATE"
    else:
        return "TEXT"

def generate_schema(df, table_name):
    """
    Generate the schema string for the given DataFrame and table name.
    
    Args:
        df (pd.DataFrame): The DataFrame to generate the schema from.
        table_name (str): The name of the table.
        
    Returns:
        str: The schema string.
    """
    schema = f"Table: {table_name}\n"
    for col in df.columns:
        dtype = df[col].dtype
        sql_type = get_sql_type(dtype)
        schema += f"- {col} ({sql_type})\n"
    return schema

def main():
    print("Welcome to the Text-to-SQL Generator!")
    csv_path = input("Please provide the path to your CSV file: ")
    
    # Read the CSV
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return
    
    # Get table name from file name
    table_name = os.path.splitext(os.path.basename(csv_path))[0]
    table_name = table_name.replace(" ", "_")  # Simple sanitization
    
    # Generate schema
    schema = generate_schema(df, table_name)
    
    # Create prompt template
    prompt_template = f"""
You are an expert SQL query generator. Given a database with the following schema:

{schema}

Please convert the following natural language query into a valid SQL query for this table:

User query: {{query}}

Return ONLY the SQL query without any explanations or additional text. The query should be complete, executable and follow best practices.

SQL Query:
"""
    
    prompt = PromptTemplate.from_template(prompt_template)
    
    # Create chain
    chain = (
        {"query": RunnablePassthrough()} 
        | prompt 
        | llm 
        | StrOutputParser()
    )
    
    print(f"Schema inferred for table '{table_name}':")
    print(schema)
    print("You can now ask questions about this dataset.")
    
    while True:
        user_query = input("Enter your question (or 'quit' to exit): ")
        if user_query.lower() == 'quit':
            break
        try:
            sql_query = chain.invoke(user_query)
            print(f"Generated SQL Query: {sql_query}")
        except Exception as e:
            print(f"Error generating SQL query: {e}")
    
    print("Thank you for using the Text-to-SQL Generator!")

if __name__ == "__main__":
    main()

Welcome to the Text-to-SQL Generator!
Error reading CSV file: [Errno 2] No such file or directory: 'exit'


In [10]:
import streamlit as st
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough
import os

# Set up Groq API key (assumes it's stored in Streamlit secrets)
os.environ["GROQ_API_KEY"] = st.secrets["GROQ_API_KEY"]

# Initialize the LLM
llm = ChatGroq(
    model_name="gemma2-9b-it",
    temperature=0,
)

# Function to map pandas dtype to SQL type
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(dtype):
        return "REAL"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATE"
    else:
        return "TEXT"

# Function to generate schema from DataFrame
def generate_schema(df, table_name):
    schema = f"Table: {table_name}\n"
    for col in df.columns:
        dtype = df[col].dtype
        sql_type = get_sql_type(dtype)
        schema += f"- {col} ({sql_type})\n"
    return schema

# Streamlit app
def main():
    st.title("Text-to-SQL Generator")

    # Initialize session state variables
    if 'df' not in st.session_state:
        st.session_state.df = None
    if 'schema' not in st.session_state:
        st.session_state.schema = None
    if 'table_name' not in st.session_state:
        st.session_state.table_name = None

    # File uploader for CSV
    uploaded_file = st.file_uploader("Upload your CSV file", type="csv")

    if uploaded_file is not None:
        try:
            # Read the CSV file
            df = pd.read_csv(uploaded_file)
            st.session_state.df = df

            # Derive table name from file name
            table_name = uploaded_file.name.split('.')[0].replace(" ", "_")
            st.session_state.table_name = table_name

            # Generate and store schema
            schema = generate_schema(df, table_name)
            st.session_state.schema = schema

            # Display the schema
            st.write(f"Schema inferred for table '{table_name}':")
            st.code(schema, language="markdown")
        except Exception as e:
            st.error(f"Error reading CSV file: {e}")

    # If schema is available, allow user to ask questions
    if st.session_state.schema:
        query = st.text_input("Enter your question:", placeholder="e.g., What are the top 10 items by sales?")

        if query:
            try:
                # Create prompt template with the schema
                prompt_template = f"""
You are an expert SQL query generator. Given a database with the following schema:

{st.session_state.schema}

Please convert the following natural language query into a valid SQL query for this table:

User query: {{query}}

Return ONLY the SQL query without any explanations or additional text. The query should be complete, executable and follow best practices.

SQL Query:
"""
                prompt = PromptTemplate.from_template(prompt_template)

                # Create the chain
                chain = (
                    {"query": RunnablePassthrough()}
                    | prompt
                    | llm
                    | StrOutputParser()
                )

                # Generate SQL query
                sql_query = chain.invoke(query)

                # Display the generated SQL query
                st.write("Generated SQL Query:")
                st.code(sql_query, language="sql")
            except Exception as e:
                st.error(f"Error generating SQL query: {e}")
    else:
        st.write("Please upload a CSV file to start.")

if __name__ == "__main__":
    main()

StreamlitSecretNotFoundError: No secrets found. Valid paths for a secrets.toml file or secret directories are: C:\Users\fahim\.streamlit\secrets.toml, d:\JMM_Technologies\Groq\.streamlit\secrets.toml

In [ ]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
import os

# Set your Groq API key
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  # Replace with your actual key

# Load the user-provided dataset
dataset_path = input("Enter the path to your CSV dataset: ")
df = pd.read_csv(dataset_path)
columns = df.columns.tolist()

# Define the prompt template with guardrails
prompt = PromptTemplate(
    input_variables=["columns", "question"],
    template="""You are an SQL expert. Given a table named "data" with columns: {columns}, 
    generate an SQL query for: "{question}". 
    Use only the provided columns. If the question can't be answered, return "Insufficient data". 
    Output only the SQL query, no explanation."""
)

# Set up the Groq model
chat_model = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="gemma2-9b-it",
    temperature=0.0  # For deterministic output
)

# Create the chain
chain = prompt | chat_model

question = input("Ask a question about the dataset: ")

# Generate and display the SQL query
response = chain.invoke({"columns": ", ".join(columns), "question": question})
sql_query = response.content.strip()

print("Generated SQL Query:")
print(sql_query)

FileNotFoundError: [Errno 2] No such file or directory: 'exit'